In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('../data/train.csv')

In [3]:
train.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,259,255,4,874724710,My Best Friend's Wedding (1997),Romance,4.0
1,259,286,4,874724727,"English Patient, The (1996)","Romance, War",4.0
2,259,298,4,874724754,Face/Off (1997),"Action, Sci-Fi, Thriller",4.0
3,259,185,4,874724781,Psycho (1960),"Horror, Romance, Thriller",4.0
4,259,173,4,874724843,"Princess Bride, The (1987)","Action, Adventure, Romance",4.0


In [24]:
train.groupby('user_id')['movie_title'].apply(list)

user_id
1      [Empire Strikes Back, The (1980), Monty Python...
5      [unknown, Star Trek: First Contact (1996), Jac...
6      [Kolya (1996), English Patient, The (1996), L....
8      [Contact (1997), Liar Liar (1997), In & Out (1...
10     [Full Monty, The (1997), L.A. Confidential (19...
                             ...                        
937    [Contact (1997), Ulee's Gold (1997), English P...
939    [Jackal, The (1997), Kull the Conqueror (1997)...
940    [English Patient, The (1996), Everyone Says I ...
941    [Contact (1997), Air Force One (1997), Liar Li...
943    [Shawshank Redemption, The (1994), Return of t...
Name: movie_title, Length: 621, dtype: object